In [1]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma

llm = OllamaLLM(model="gemma2")
output_parser = StrOutputParser()
embedding_model = OllamaEmbeddings(model='gemma2')

# 카드 데이터베이스 경로
DB_PATH = '/workspace/card_db/vector'

# Chroma 데이터베이스 로드
card_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embedding_model,
    collection_name='card_db',
)

In [2]:
# 사용자가 보유한 카드 정보
owned_cards = ['K-패스 신한카드', '신한카드 Deep Oil' '신한카드 Air One']

In [18]:
# 초기 결제 상황 정보

# # Case 1.
# environment = '오프라인 매장'
# store = '세븐일레븐'
# date = '2024년 11월 21일 목요일'
# time = '오후 9시 10분'

# # Case 2.
# environment = '오프라인 매장'
# store = '현대오일뱅크'
# date = '2024년 11월 22일 금요일'
# time = '오후 6시 50분'

# Case 3.
environment = '오프라인 매장'
store = '메가MGC커피'
date = '2024년 11월 23일 토요일'
time = '오전 10시 30분'

In [19]:
# Step 2: 카테고리 추론 프롬프트와 체인 구성
category_prompt_template = """
네 역할은 경영 전문가야.

주어진 결제 상황을 보고 어떤 유형의 매장에서 결제가 발생할 것인지 추론해.
결제 상황:
- 온/오프라인: {environment}
- 매장: {store}
- 날짜: {date}
- 시간: {time}

결과는 입력된 매장에 대한 유형을 단답식으로 출력해.
출력 예시는 다음과 같아.
커피전문점
음식점
편의점
학원
주유소
전자제품 판매점
영화관
"""

category_prompt = PromptTemplate(
    input_variables=['environment', 'store', 'date', 'time'],
    template=category_prompt_template
)

category_chain = category_prompt | llm | output_parser
category = category_chain.invoke({'environment': environment, 'store': store, 'date': date, 'time': time}).strip()
category

'커피 전문점'

In [24]:
# Step 3: 예상 결제액 추론 프롬프트와 체인 구성
estimation_prompt_template = """
네 역할은 금융 전문가야.

주어진 결제 상황과 카테고리에 따라 예상되는 결제 금액을 추정해.
결제 상황:
- 온/오프라인: {environment}
- 매장: {store}
- 날짜: {date}
- 시간: {time}
- 업종: {category}

결과는 숫자로만 출력해.
출력 예시는 다음과 같아.
1000
5000
30000
75000
"""

estimation_prompt = PromptTemplate(
    input_variables=['environment', 'store', 'date', 'time', 'category'],
    template=estimation_prompt_template
)

estimation_chain = estimation_prompt | llm | output_parser
estimated_amount = estimation_chain.invoke({'environment': environment, 'store': store, 'date': date, 'time': time, 'category': category}).strip()
estimated_amount

'6000'

In [16]:
from langchain.chains import RetrievalQA

# RAG 구성
# retrieval_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type='stuff',  # 기본적인 RAG 방식
#     retriever=card_db.as_retriever(),
#     return_source_documents=True  # 원본 문서도 반환하도록 설정
# )

owned_cards_query = f"{', '.join(owned_cards)}의 카드 정보를 알려줘."
retriever = card_db.as_retriever()
owned_cards_info = retriever.get_relevant_documents(query=owned_cards_query)

# Step 4: 최적의 카드 추론 프롬프트와 체인 구성
card_prompt_template = """
네 역할은 금융 전문가야.

다음 결제 상황을 고려하여 사용자가 가진 카드 중에서 혜택이 가장 좋은 카드를 추천해줘.
결제 상황:
- 온/오프라인: {environment}
- 매장: {store}
- 날짜: {date}
- 시간: {time}
- 업종: {category}
- 예상 결제액: {estimated_amount}
- 사용자가 소유한 카드 정보: {owned_cards_info}

혜택이 가장 좋은 카드를 반드시 하나 알려줘.
출력 예시는 다음과 같아.
국민행복
Mr.Life
처음(ANNIVERSE)
"""

card_prompt = PromptTemplate(
    input_variables=['environment', 'store', 'date', 'time', 'category', 'estimated_amount', 'owned_cards_info'],
    template=card_prompt_template
)

output_parser = StrOutputParser()
card_chain = (card_prompt | llm | output_parser)

# 카드 추천 체인 실행
result = card_chain.invoke({
    'environment': environment,
    'store': store,
    'date': date,
    'time': time,
    'category': category,
    'estimated_amount': estimated_amount,
    'owned_cards_info': owned_cards_info
}).strip()

# 최종 결과 출력
print('혜택이 가장 좋은 카드:', result)

혜택이 가장 좋은 카드: 사용자분께서는 영화관에서 결제를 하는 상황이므로, **K-패스** 신한카드가 가장 적합합니다. 

K-패스 신한카드는 오프라인 매장에서 영화 관람 시 5%의 결제일 할인을 제공하며,  롯데시네마 등은 K-패스 카드에서 제공하는 할인 대상 업종으로 분류됩니다.
